In [13]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1caf7c69a1e046cbb74de6812fc975d6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='3Bwomte_QO1hWqYGiEOgj-2zC4jwZmPB16VmZTF7imyl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_1caf7c69a1e046cbb74de6812fc975d6.get_object(Bucket='machinelearningdicoding-donotdelete-pr-ypqn86qbi4dqfk',Key='Social_Network_Ads.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [32]:
# melihat nilai kosong
# df.info()

# menghapus atribut tak penting untuk dipelajari oleh model
data = df.drop(columns=['User ID'])

# membuat data dummies dari dataset
data = pd.get_dummies(data)
data

,Age,EstimatedSalary,Purchased,Gender_Female,Gender_Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1
...,...,...,...,...,...
395,46,41000,1,1,0
396,51,23000,1,0,1
397,50,20000,1,1,0
398,36,33000,0,0,1


In [39]:
# pisahkan atribut dan label / class
predictions = ['Age' , 'EstimatedSalary' , 'Gender_Female' , 'Gender_Male']
x = data[predictions]
y = data['Purchased']

In [42]:
# melakukan data preparation standardization
# y
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
scaled_data = scaler.transform(x)

scaled_data = pd.DataFrame(scaled_data, columns=x.columns)
scaled_data.head()

,Age,EstimatedSalary,Gender_Female,Gender_Male
0,-1.781797,-1.490046,-1.020204,1.020204
1,-0.253587,-1.460681,-1.020204,1.020204
2,-1.113206,-0.785290,0.980196,-0.980196
3,-1.017692,-0.374182,0.980196,-0.980196
4,-1.781797,0.183751,-1.020204,1.020204


In [44]:
from sklearn.model_selection import train_test_split

# bagi data menjadi train dan test untuk setiap atribut dan label
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=1)

In [45]:
# membuat model

from sklearn import linear_model

# latih model dengan fungsi fit
model = linear_model.LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [46]:
# uji akurasi model pada test set dengan fungsi score()

from sklearn.preprocessing import StandardScaler

# uji akurasi model
model.score(x_test, y_test)

0.825